# 문서 처리를 위한 의미적 분할 (Semantic Chunking)

## 개요

이 코드는 PDF 문서를 처리하고 정보를 검색하기 위한 의미적 분할 접근법을 구현합니다. 고정된 문자 또는 단어 수에 따라 텍스트를 분할하는 기존의 방법과는 달리, 의미적 분할은 더 유의미하고 맥락에 맞는 텍스트 세그먼트를 만드는 것을 목표로 합니다.

## 필요성

기존의 텍스트 분할 방법은 종종 임의의 지점에서 텍스트를 분할하여 정보와 맥락의 흐름을 방해할 가능성이 있습니다. 이러한 문제를 해결하기 위해 의미적 분할은 더 자연스러운 구분점에서 텍스트를 분할하려고 시도하며 각 청크 내에서 의미론적 일관성을 유지합니다.

## 주요 구성 요소

1. PDF 처리 및 텍스트 추출
2. LangChain의 SemanticChunker를 사용한 의미적 분할
3. FAISS 및 SDS 임베딩을 사용하여 벡터 저장소 만들기
4. 처리된 문서를 쿼리하기 위한 검색기 설정

## 방법 세부 정보

### 문서 전처리

1. PDF가 읽히고 사용자 지정 `read_pdf_to_string` 함수를 사용하여 문자열로 변환됩니다.

### 의미적 분할 (Semantic Chunking)

1. SDS 임베딩을 사용하는 LangChain의 `SemanticChunker`를 활용합니다.
2. 3가지 유형의 분할점이 제공됩니다.
   - '백분위수': X 백분위수보다 큰 차이에서 분할합니다.
   - '표준편차': X 표준 편차 이상에서 분할합니다.
   - '사분위간 거리'를 사용하여 분할 지점을 결정합니다.
3. 이 구현에서는 90의 임계값과 함께 '백분위수' 방법이 사용됩니다.

### 벡터 스토어 만들기

1. SDS 임베딩은 의미적 청크의 벡터 표현을 만드는 데 사용됩니다.
2. 효율적인 유사도 검색을 위해 이러한 임베딩으로 FAISS 벡터 저장소를 만듭니다.

### 검색기 설정

1. 주어진 쿼리에 대해 가장 관련성이 높은 청크 상위 2개를 가져오도록 검색기가 구성됩니다.

## 주요 기능

1. 맥락 인식 분할: 청크 내에 완전한 생각이나 아이디어를 포함하도록 노력합니다.
2. 유연한 구성: 다른 분기점 유형과 임계값을 사용할 수 있습니다.
3. 고급 NLP 도구와의 통합: 의미적 분할 및 검색 모두에 SDS 임베딩을 사용합니다.

## 이 접근법의 이점

1. 향상된 일관성: 청크는 보다 완전하고 논리적인 생각을 포함할 가능성이 높습니다.
2. 더 나은 검색 정확도: 맥락을 보존함으로써 검색 정확도가 향상될 수 있습니다.
3. 적응성: 문서의 특성과 검색 요구 사항에 따라 분할 방법을 조정할 수 있습니다.
4. 잠재적으로 더 나은 이해: 더 일관성 있는 텍스트 세그먼트가 있으면 LLM 또는 다운스트림 작업이 더 잘 수행될 수 있습니다.

## 구현 세부 정보

1. Semantic Chunking 프로세스와 최종 벡터 표현 모두에 SDS의 임베딩을 사용합니다.
2. 효율적인 검색 가능한 인덱스를 만들기 위해 FAISS를 사용합니다.
3. 검색기는 필요에 따라 조정할 수 있는 가장 관련성이 높은 청크 상위 2개를 반환하도록 설정되어 있습니다.

## 예시 사용법

코드에는 "기후 변화의 주요 원인은 무엇입니까?"라는 테스트 쿼리가 포함되어 있습니다. 이것은 처리된 문서에서 관련 정보를 찾는 데 의미적 분할 및 검색 시스템이 어떻게 사용될 수 있는지를 보여줍니다.

## 결론

Semantic Chunking은 검색 시스템에 대한 문서 처리의 고급 접근 방식을 나타냅니다. 텍스트 세그먼트 내에서 의미론적 일관성을 유지하려고 시도함으로써 검색된 정보의 품질을 개선하고 다운스트림 NLP 작업의 성능을 높일 수 있는 잠재력이 있습니다. 이 기법은 과학적 논문, 법률 문서 또는 종합 보고서처럼 맥락 유지가 중요한 긴 복잡한 문서를 처리하는 데 특히 유용합니다.

### Import libraries 

In [1]:
%pip install -qU langchain_experimental sentence_transformers langchain_community text_generation fitz PyMuPDF faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks

from langchain_experimental.text_splitter import SemanticChunker
from helper_functions import *

### Define file path

In [3]:
path = "../data/Understanding_Climate_Change.pdf"

### Read PDF to string

In [4]:
content = read_pdf_to_string(path)

### Breakpoint types: 
* 'percentile': all differences between sentences are calculated, and then any difference greater than the X percentile is split.
* 'standard_deviation': any difference greater than X standard deviations is split.
* 'interquartile': the interquartile distance is used to split chunks.

In [5]:
from typing import (List)

import requests
from langchain_core.embeddings import Embeddings

class CustomEmbedding(Embeddings):
    def __init__(self):
        self.embed_url = 'http://sds-embed.serving.70-220-152-1.sslip.io/v1/models/embed:predict'

    def call_embed(self, url, texts):
        data = {
            "instances": texts
        }
        response = requests.post(url=url, json=data)
        result = response.json()
        return result['predictions']

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        주어진 텍스트를 임베딩하여 벡터로 반환 합니다.
        """
        embed_list = self.call_embed(url=self.embed_url, texts=texts)
        return embed_list

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""

        embed_list = self.call_embed(url=self.embed_url, texts=[text])
        return embed_list[0]

In [6]:
text_splitter = SemanticChunker(CustomEmbedding(), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use

### Split original text to semantic chunks

In [7]:
docs = text_splitter.create_documents([content])

### Create vector store and retriever

In [8]:
from langchain.vectorstores import FAISS

embeddings = CustomEmbedding()
vectorstore = FAISS.from_documents(docs, embeddings)
chunks_query_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

### Test the retriever

In [9]:
test_query = "What is the main cause of climate change?"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

Context 1:
Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past century, human 
activities, particularly the burning of fossil fuels and deforestation, have significantly 
contributed to climate change. Historical Context 
The Earth's climate has changed throughout history. Over the past 650,000 years, there have 
been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 
11,700 years ago marking the beginning of the modern climate era and human civilization. Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which 
began at the end of the last ice age, huma

/home/user/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
